# Read in Ordered and Unordered Transfer Values

In [64]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [9]:
unordered_pivot = pd.read_csv('unordered_pivot.csv',index_col=0)
transfers = pd.read_csv('unclipped_transfer_table.csv')

# Compute Support with Order

In [10]:
transfers[(transfers['Recipient'] == 'CpY11') & (transfers['Donor'] == 'LYRa11')]

,Donor,Recipient,Gene Family,Support
4,LYRa11,CpY11,ORF7a,1000
95,LYRa11,CpY11,ORF8,448
252,LYRa11,CpY11,ORF7b,200
262,LYRa11,CpY11,ORF6,200
646,LYRa11,CpY11,M,79
755,LYRa11,CpY11,E,60


In [11]:
transfers[(transfers['Donor'] == 'CpY11') & (transfers['Recipient'] == 'LYRa11')]

,Donor,Recipient,Gene Family,Support
26,CpY11,LYRa11,ORF6,800
107,CpY11,LYRa11,ORF8,415
887,CpY11,LYRa11,M,44


In [16]:
unordered_pivot

,ORF1ab,S,ORF3a,E,M,ORF6,ORF7a,ORF7b,ORF8,N,ORF10,Total
CpY11_LYRa11,0.0,0.0,0.0,60.0,123.0,1000.0,1000.0,200.0,863.0,0.0,0.0,3246.0
RpS11_n44,0.0,0.0,900.0,100.0,0.0,1000.0,0.0,800.0,15.0,0.0,0.0,2815.0
HeB2013_Jiyuan_84,1000.0,1000.0,0.0,0.0,0.0,0.0,151.0,0.0,46.0,500.0,65.0,2762.0
Rp3_RpS11,0.0,0.0,1000.0,0.0,1000.0,20.0,0.0,0.0,481.0,0.0,0.0,2501.0
YNLF_31C_n40,0.0,900.0,900.0,0.0,672.0,0.0,0.0,0.0,0.0,0.0,0.0,2472.0
...,...,...,...,...,...,...,...,...,...,...,...,...
BM48-31_n26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0
YN2018A_n18,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
Rs4084_n9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
n2_n6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [95]:
unordered = unordered_pivot.reset_index().iloc[:,:-1].melt(id_vars=['index'])
unordered
unordered = unordered.set_index('index')
unordered.columns = ['Gene Family','Support']
unordered = unordered[unordered['Support'] > 0]
unordered['Forward'] = 0
unordered['Backward'] = 0

In [96]:
unordered.head()

,Gene Family,Support,Forward,Backward
index,,,,
HeB2013_Jiyuan_84,ORF1ab,1000.0,0,0
Rs7327_Rs9401,ORF1ab,240.0,0,0
Rs4231_Rs4247_147,ORF1ab,417.0,0,0
Rs4231_n21,ORF1ab,1000.0,0,0
HuB2013_Rm1,ORF1ab,1000.0,0,0


In [97]:
for i, row in tqdm(enumerate(unordered.iterrows()),total=len(unordered)):
    first = '_'.join(row[0].split('_')[:1])
    second = '_'.join(row[0].split('_')[1:])
    gf = row[1]['Gene Family']
    forward = transfers[(transfers['Donor'] == first) &\
                        (transfers['Recipient'] == second) &\
                        (transfers['Gene Family'] == gf)]
    if len(forward):
        forward = forward['Support'].item()
    else:
        forward = 0
    unordered.iloc[i,2] = forward
    backward = transfers[(transfers['Recipient'] == first) &\
                         (transfers['Donor'] == second) &\
                         (transfers['Gene Family'] == gf)]
    if len(backward):
        backward = backward['Support'].item()
    else:
        backward = 0
    unordered.iloc[i,3] = backward

  0%|          | 0/1170 [00:00<?, ?it/s]

In [98]:
unordered['Forward Pct'] = unordered['Forward'] / (unordered['Forward'] + unordered['Backward']).fillna(0)

In [99]:
bidirectional = unordered[(unordered['Forward Pct'] > 0.4) &\
                          (unordered['Forward Pct'] < 0.6)]

In [100]:
bidirectional

,Gene Family,Support,Forward,Backward,Forward Pct
index,,,,,
HeB2013_Jiyuan_84,ORF1ab,1000.0,467,533,0.467000
Rs4231_n21,ORF1ab,1000.0,417,583,0.417000
HuB2013_Rm1,ORF1ab,1000.0,480,520,0.480000
n2_n35,ORF1ab,1000.0,537,463,0.537000
C020_Rp3,ORF1ab,838.0,400,438,0.477327
...,...,...,...,...,...
BM48-31_n34,ORF10,47.0,21,26,0.446809
Rs4231_n35,ORF10,43.0,25,18,0.581395
HeB2013_n6,ORF10,38.0,19,19,0.500000


In [101]:
bidirectional[bidirectional['Support'] >= 100]

,Gene Family,Support,Forward,Backward,Forward Pct
index,,,,,
HeB2013_Jiyuan_84,ORF1ab,1000.0,467,533,0.467000
Rs4231_n21,ORF1ab,1000.0,417,583,0.417000
HuB2013_Rm1,ORF1ab,1000.0,480,520,0.480000
n2_n35,ORF1ab,1000.0,537,463,0.537000
C020_Rp3,ORF1ab,838.0,400,438,0.477327
...,...,...,...,...,...
n44_n5,ORF10,144.0,71,73,0.493056
BM48-31_n35,ORF10,132.0,66,66,0.500000
n23_n47,ORF10,100.0,46,54,0.460000
